In [186]:
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
import dash_table
import networkx as nx
import pandas as pd
import numpy as np
import json
import pickle 

from app import app

# needed only if running this as a single page app
#external_stylesheets = [dbc.themes.LUX]

#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# change to app.layout if running as single page app instead

graph_style = {"maxHeight": "auto", "align":"center", "margin": "auto"}

# Read data
df = pd.read_csv('HP_enriched_character_df.csv')

with open("G.pickle", 'rb') as f:
    G = pickle.load(f)

with open("GCC.pickle", 'rb') as f:
    GCC = pickle.load(f)


In [187]:
in_degree_vals = list(nx.get_node_attributes(G,'in_degree').values())
out_degree_vals = list(nx.get_node_attributes(G,'out_degree').values())

# Get hist values
hist_in, bins_in = np.histogram(in_degree_vals, bins = np.arange(np.min(in_degree_vals), np.max(in_degree_vals) + 2))
centered_in = (bins_in[:-1] + bins_in[1:]) / 2
hist_out, bins_out = np.histogram(out_degree_vals, bins = np.arange(np.min(out_degree_vals), np.max(out_degree_vals) + 2))
centered_out = (bins_out[:-1] + bins_out[1:]) / 2

df_in_degree = pd.DataFrame({"In-degree": hist_in, "Count": centered_in})
df_out_degree = pd.DataFrame({"Out-degree": hist_out, "Count": centered_out})

# Make figure
fig = go.Figure(make_subplots(rows=2, cols=1))

marker_layout_in = dict(size=8,color = "blue", line_width=1,opacity = 0.5)
marker_layout_out = dict(size=8,color = "red", line_width=1,opacity = 0.5)

# Add traces
fig.add_trace(go.Scatter(x=df_in_degree["Count"], y=df_in_degree["In-degree"],
                    mode='markers',
                    marker=marker_layout_in,
                    hoverinfo='none',
                    name='In-degree'), 
            row=1, col=1
            )

fig.add_trace(go.Scatter(x=df_out_degree["Count"], y=df_out_degree["Out-degree"],
                    mode='markers',
                    marker=marker_layout_out,
                    hoverinfo='none',
                    name='Out-degrees'),
            row=1, col=1
            )


fig.add_trace(go.Scatter(x=df_in_degree["Count"], y=df_in_degree["In-degree"],
                mode='markers',
                    marker=marker_layout_in,
                    hoverinfo='none',
                    name='In-degree',
                    showlegend  =False), 
            row=2, col=1
            )

fig.add_trace(go.Scatter(x=df_out_degree["Count"], y=df_out_degree["Out-degree"],
                    mode='markers',
                    marker=marker_layout_out,
                    hoverinfo='none',
                    name='Out-degrees',
                    showlegend  =False),
            row=2, col=1
            )

# Update xaxis properties
fig.update_xaxes(title_text="Degree", row=1, col=1)
fig.update_xaxes(title_text="log(Degree)", row=2, col=1, type="log")

# Update yaxis properties
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="log(Count)", row=2, col=1, type="log")

# Update title and height
fig.update_layout(title_text="Degree distribution", height=800)

fig.update_layout(
    legend=dict(
        font=dict(
            #family="Courier",
            size=18,
            color="black"
        )),
    title=dict(
        font=dict(
            #family="Courier",
            size=20,
            color="black"
        )
    )
    )

## To cyto file

In [188]:
# Update positions
pos = nx.get_node_attributes(G,'pos')
pos = {i: np.array(p) for i, p in pos.items()}
nx.set_node_attributes(G,pos, 'pos')

pos = nx.get_node_attributes(GCC,'pos')
pos = {i: np.array(p) for i, p in pos.items()}
nx.set_node_attributes(GCC,pos, 'pos')

In [189]:
# Update positions (scale)
positions = nx.get_node_attributes(G,'pos')
positions.update((key, tuple([10000*x for x in value])) for key, value in positions.items())
nx.set_node_attributes(G, positions, 'pos')

positions = nx.get_node_attributes(GCC,'pos')
positions.update((key, tuple([10000*x for x in value])) for key, value in positions.items())
nx.set_node_attributes(GCC, positions, 'pos')

# Update size (scale)
positions = nx.get_node_attributes(G,'size')
positions.update((key, value) for key, value in positions.items())
nx.set_node_attributes(G, positions, 'size')

positions = nx.get_node_attributes(GCC,'size')
positions.update((key, value) for key, value in positions.items())
nx.set_node_attributes(GCC, positions, 'size')

In [190]:
cyto_GCC_json = nx.cytoscape_data(GCC)['elements']

for node in cyto_GCC_json['nodes']:
    node.update({'classes': node['data']['house']})
    node.update({'position': {"x":node['data']['pos'][0],"y":node['data']['pos'][1]}})
    node.update({'size': node['data']['size']})
    
import json
with open('cyto_GCC.json', 'w') as outfile:
       json.dump(cyto_GCC_json, outfile)
        
cyto_full_network_json = nx.cytoscape_data(G)['elements']

for node in cyto_full_network_json['nodes']:
    node.update({'classes': node['data']['house']})
    node.update({'position': {"x":node['data']['pos'][0],"y":node['data']['pos'][1]}})
    node.update({'size': node['data']['size']})
                
import json
with open('cyto_full_network.json', 'w') as outfile:
       json.dump(cyto_full_network_json, outfile)

In [191]:
with open("cyto_full_network.json", 'rb') as f:
    cyto_G = json.load(f)

with open("cyto_GCC.json", 'rb') as f:
    cyto_GCC = json.load(f)



In [193]:
cyto_G['nodes'][1]['position']

{'x': -1424.1233995045666, 'y': -1491.447309906207}

In [1]:
import nltk
import nltk.book as book
text1 = book.text1

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [9]:
print(text1[0:100])

['[', 'Moby', 'Dick', 'by', 'Herman', 'Melville', '1851', ']', 'ETYMOLOGY', '.', '(', 'Supplied', 'by', 'a', 'Late', 'Consumptive', 'Usher', 'to', 'a', 'Grammar', 'School', ')', 'The', 'pale', 'Usher', '--', 'threadbare', 'in', 'coat', ',', 'heart', ',', 'body', ',', 'and', 'brain', ';', 'I', 'see', 'him', 'now', '.', 'He', 'was', 'ever', 'dusting', 'his', 'old', 'lexicons', 'and', 'grammars', ',', 'with', 'a', 'queer', 'handkerchief', ',', 'mockingly', 'embellished', 'with', 'all', 'the', 'gay', 'flags', 'of', 'all', 'the', 'known', 'nations', 'of', 'the', 'world', '.', 'He', 'loved', 'to', 'dust', 'his', 'old', 'grammars', ';', 'it', 'somehow', 'mildly', 'reminded', 'him', 'of', 'his', 'mortality', '.', '"', 'While', 'you', 'take', 'in', 'hand', 'to', 'school', 'others', ',']


In [18]:
c = nltk.ConcordanceIndex(text1.tokens, key = lambda s: s.lower())
print([text1.tokens[offset+i] for offset in c.offsets('to') for i in range(1,5)])

['a', 'Grammar', 'School', ')', 'dust', 'his', 'old', 'grammars', 'school', 'others', ',', 'and', 'teach', 'them', 'by', 'what', 'be', 'called', 'in', 'our', 'roll', ',', 'to', 'wallow', 'wallow', '."', '--', 'RICHARDSON', 'have', 'gone', 'through', 'the', 'whales', 'he', 'could', 'anyways', 'that', 'hopeless', ',', 'sallow', 'sit', ',', 'and', 'feel', 'them', 'bluntly', ',', 'with', 'please', 'the', 'world', ',', 'the', 'royal', '-', 'mast', 'shine', 'after', 'him', ';', 'be', 'hoary', '."', '--', 'swallow', 'up', 'Jonah', '."', 'play', 'therein', '."', '--', 'the', 'king', '.', '...', 'seethe', 'like', 'boiling', 'pan', 'secure', ',', 'no', 'skill', 'returne', 'againe', 'To', 'his', 'his', 'wound', "'", 's', 'shore', 'flies', 'thro', "'", 'GONDIBERT', '.', '"', 'What', 'attend', 'their', 'prey', ',', 'SPITZBERGEN', ',', 'IN', 'PURCHAS', 'proceed', 'with', 'a', 'great', 'see', 'whether', 'they', 'can', 'GREENLAND', ',', 'A', '.', 'try', 'whether', 'I', 'can', 'one', ';', 'than', 'we',